In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba 
from gensim.models import Word2Vec
from gensim.corpora.dictionary import Dictionary
import multiprocessing
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split 
from sklearn.utils import shuffle 
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout,Activation
from keras.models import model_from_yaml
from Attention import attention
%matplotlib inline

D:\Program Files (x86)\anaconda\envs\reputation\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\Program Files (x86)\anaconda\envs\reputation\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


ImportError: No module named 'MyAttention'

In [ ]:
# set parameters(word2vec)
vocab_dim = 200
maxlen = 100
n_iterations = 1  # ideally more..
n_exposures = 10
window_size = 7
cpu_count = multiprocessing.cpu_count()

In [ ]:
data=pd.read_csv('data/train_first.csv')
#data=data.drop(['Id'],axis=1)
#data.head()
data=data.sample(frac=1).reset_index(drop=True) 

#划分x  y
X=data['Discuss']
Y=data['Score']
 
#y ONEHOT
lb = preprocessing.LabelBinarizer()
lb.fit([1,2,3,4,5])
Y=lb.transform(Y)

In [ ]:
X[0]

In [ ]:
#加载停用词
def get_stopwords(path):
    return [line.strip() for line in open(path,'r',encoding='utf-8').readlines()]
#句子去停用词
def removestopwords(sentence):
        stopwords_list=get_stopwords('data/stopwords.txt')
        outstr=[]
        for word in sentence:
            if not word in stopwords_list:
                if word!='\n' and word!='\t':
                     outstr.append(word)
        return outstr

In [ ]:
#分词 并去掉停用词
def cut(sentence):
    return removestopwords(jieba.cut(sentence))
#分词后的word
cabs=[cut(x) for x in X]
#cabs[0]

In [ ]:
#word2vec

#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def word2vec_train(combined):

    model = Word2Vec(size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=cpu_count,
                     iter=n_iterations)
    model.build_vocab(combined)
    model.train(combined,total_examples=model.corpus_count,epochs=model.iter)
    model.save('data/Word2vec.pkl')
    index_dict, word_vectors,combined = create_dictionaries(model=model,combined=combined)
    return   index_dict, word_vectors,combined


#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def create_dictionaries(model=None,
                        combined=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.wv.vocab.keys(),
                            allow_update=True)
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量

        def parse_dataset(combined):
            ''' Words become integers
            '''
            data=[]
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0)
                data.append(new_txt)
            return data
        combined=parse_dataset(combined)
        combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec,combined
    else:
        print ('No data provided...')

In [ ]:
#根据词向量重新填充x
index_dict, word_vectors,combined=word2vec_train(cabs)


In [ ]:
#搭建网络结构
input_x=tf.placeholder(shape=[None,None],dtype=tf.int32,name='input_x')
label=tf.placeholder(shape=[None,None],dtype=tf.int32,name='target')
lr=tf.placeholder(dtype=tf.float32,name='learning_rate')
#sequence_lengths=tf.placeholder(shape=[None],dtype=tf.int32,name='sequence_lengths')
keep_prob=tf.placeholder(dtype=tf.float32,name='keep_prob')
#embedding_placeholder = tf.placeholder(tf.float32, [None, None],name='embedding_placeholder')
#super params
hidden_dim=128
layer_num=2
attention_size=100

#bilstm
def get_lstm_dropout():
    lstm_cell=tf.contrib.rnn.BasicLSTMCell(hidden_dim,forget_bias=1.0)
    return tf.contrib.rnn.DropoutWrapper(cell=lstm_cell,output_keep_prob=keep_prob)
#fw
fw_cells=tf.contrib.rnn.MultiRNNCell([get_lstm_dropout() for _ in range(layer_num)])
initial_state_fw =fw_cells.zero_state(tf.shape(input_x)[0],tf.float32)
#bw
bw_cells=tf.contrib.rnn.MultiRNNCell([get_lstm_dropout() for _ in range(layer_num)])
initial_state_bw=bw_cells.zero_state(tf.shape(input_x)[0],tf.float32)
#embedding
embedding = tf.Variable(tf.random_uniform((len(index_dict)+1,vocab_dim), -1, 1),name='embedding')
embed = tf.nn.embedding_lookup(embedding, input_x)
#
outputs,_,_=tf.nn.bidirectional_dynamic_rnn(fw_cells,bw_cells,embed,initial_state_fw=initial_state_fw,
    initial_state_bw=initial_state_bw,dtype=tf.float32)

#attention mechanism
attention_output,alphas= attention(outputs, attention_size, return_alphas=True)

#with tf.variable_scope('optmizer'):
predictions=tf.contrib.layers.fully_connected(attention_output[:,-1],5,activation_fn=tf.nn.softmax)
cost=tf.losses.mean_squared_error(label,predictions)
optimizer=tf.train.AdamOptimizer(lr).minimize(cost)
#validation accuracy
correct_pred=tf.equal(tf.cast(tf.round(predictions),tf.int32),label)
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [ ]:
#batching
def get_batching(x,y,batch_size=100):
    n_batches=len(x)//batch_size
    x,y=x[:n_batches*batch_size],y[:n_batches*batch_size]
    for ii in range(0,len(x),batch_size):
        yield x[ii:ii+batch_size],y[ii:ii+batch_size]        

In [ ]:
#训练
#super params
epochs=10
batch_size=100
saver=tf.train.Saver()
with tf.Session()as sess:
    writer = tf.summary.FileWriter("logs/", sess.graph)
    sess.run(tf.global_variables_initializer())
    x_train, x_test, y_train, y_test=train_test_split(combined, Y, test_size=0.2)#get_data(index_dict,word_vectors,combined,Y)
    iteration = 1
    for e in range(epochs):
        for ii, (x, y) in enumerate(get_batching(x_train, y_train, batch_size), 1):
            feed={input_x:x,label:y,lr:0.001,keep_prob:0.5}
            loss,_=sess.run([cost,optimizer],feed_dict=feed)
            tf.summary.scalar('loss',loss)
            if iteration%50==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
            if iteration%100==0:
                val_acc = []
                for x, y in get_batching(x_test, y_test, batch_size):
                    feed = {input_x: x,
                            label: y,                            
                            keep_prob: 1}
                    batch_acc= sess.run(accuracy, feed_dict=feed)
                    val_acc.append(batch_acc)
                tf.summary.scalar('accuracy',np.mean(val_acc))
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

In [ ]:
predict_data=pd.read_csv('data/predict_first.csv')
cabs=[cut(x) for x in predict_data['Discuss']]
_,_,predict_X=word2vec_train(cabs)

In [ ]:
saver=tf.train.Saver()
model_file=tf.train.latest_checkpoint('checkpoints/')
with tf.Session()as sess:
    #sess.run(tf.global_variables_initializer()) 
    saver.restore(sess,model_file)
    predict_arr=[]
    for ii in range(0,len(predict_X),100):
        for p in sess.run(predictions,feed_dict={input_x:predict_X[ii:ii+100],keep_prob:1.0}):
            predict_arr.append(np.argmax(p)+1)
    

In [ ]:
len(predict_arr)